# Handling data imbalance (classification)

- Import the required libraries and modules that you would need.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import warnings
warnings.filterwarnings('ignore')

- Read that data into Python and call the dataframe churnData.

In [2]:
churnData = pd.read_csv("./files_for_lab/Customer-Churn.csv")
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
churnData.shape

(7043, 16)

- Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

In [4]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors = 'coerce')

In [6]:
churnData["TotalCharges"].dtypes

dtype('float64')

- Check for null values in the dataframe. Replace the null values.

In [7]:
churnData.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [8]:
churnData['TotalCharges'].fillna((churnData['TotalCharges'].mean()), inplace=True)

In [9]:
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

- Scale the features (tenure, SeniorCitizen, MonthlyCharges and TotalCharges) either by using normalizer or a standard scaler.

In [10]:
y = churnData['Churn']

In [11]:
X = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]

In [12]:
transformer = Normalizer() 
transformer.fit(X)
x_normalized = transformer.transform(X)

- Split the data into a training set and a test set.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

- Fit a logistic regression model on the training data.

In [14]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

- Check the accuracy on the test data.

In [15]:
logreg.score(X_test, y_test)

0.7825099375354913

### Imbalance management

- Check for the imbalance.

In [16]:
y.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

- Use upsampling and downsampling to create a balance between the two classes. Each time fit the model and see how the accuracy of the model is.

#### Upsample l first method

In [17]:
data = pd.concat([X_train, y_train], axis=1)

In [18]:
churn_no = data[data['Churn'] == 'No'].sample(5174, replace=True)
churn_yes = data[data['Churn'] == 'Yes'].sample(5174, replace=True) 

In [19]:
churn_no = churn_no.sample(len(churn_no), replace=True)

In [20]:
data = pd.concat([churn_no, churn_yes], axis=0)
data = data.sample(frac=1)
data['Churn'].value_counts()

Yes    5174
No     5174
Name: Churn, dtype: int64

In [21]:
upsampled_train = pd.concat([churn_yes, churn_no]).sample(frac=1)
other = upsampled_train.drop("Churn", axis = 1)
target = upsampled_train["Churn"]

In [22]:
logreg.fit(other, target)
logreg.score(X_test, y_test)

0.6842703009653606

#### Upsample l second method: SMOTE

In [23]:
smote = SMOTE()

X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

Yes    5174
No     5174
Name: Churn, dtype: int64

In [24]:
logreg.fit(X_sm, y_sm)
logreg.score(X_test, y_test)

0.7137989778534923

#### Downsample l first method

In [25]:
data1 = pd.concat([X_train, y_train], axis=1)

In [26]:
No = data1[data1['Churn'] == 'No'].sample(5174, replace=True)
Yes = data1[data1['Churn'] == 'Yes'].sample(5174, replace=True)

In [27]:
downsampled = pd.concat([Yes, No]).sample(frac=1)
X_dw_train = downsampled.drop("Churn", axis = 1)
y_dw_train = downsampled["Churn"]

In [28]:
logreg.fit(X_dw_train, y_dw_train)
logreg.score(X_test, y_test)

0.7030096536059057

#### Downsample l second method: Tomek Links 

In [29]:
tomek = TomekLinks()
X_tl, y_tl = tomek.fit_resample(X, y)
y_tl.value_counts()

No     4620
Yes    1869
Name: Churn, dtype: int64

In [30]:
logreg.fit(X_tl, y_tl)
logreg.score(X_test, y_test)

0.7649063032367973